In [40]:
from bs4 import BeautifulSoup as bs
import requests 
import pandas as pd
from urllib.parse import urljoin

In [53]:
df_artists = pd.read_csv("../data/spotify_top_3000_artists_with_song_counts.csv", sep= ',')
df_artists.head(2)

,Unnamed: 0.1,Unnamed: 0,Artist,url_songs,Num songs
0,0,0,Drake,https://kworb.net/spotify/artist/3TVXtAsR1Inum...,482.0
1,1,1,Taylor Swift,https://kworb.net/spotify/artist/06HL4z0CvFAxy...,497.0


In [54]:
df_artists = df_artists[df_artists["Num songs"]>0]

In [57]:
df_artists = df_artists.reset_index(drop=True)

WRITE A FUNCTION To SCRAPE EACH URL FOR EACH ARTIST

In [42]:
def scrape_songs_url(url_song):
    page = requests.get(url_song)
    soup = bs(page.content)
    tags = soup.find_all(True)
    not_found_tag = [i for i, tag in enumerate(tags) if "404 Not Found" in tag.text]
    if not_found_tag != []:
        return 0, {}
    possible_tag_numbers = [i for i, tag in enumerate(tags) if "Song Title" in tag.text]
    tag_number = possible_tag_numbers[-4]
    elem = tags[tag_number]
    item = list(elem.children)[3]
    song_tags = bs(str(item)).find_all("a",  href=True)
    song_count = len(song_tags)

    song_spotify_id_dict = {}

    for i, song_tag in enumerate(song_tags):
        song_name = song_tag.text
        song_spotify_url = song_tag["href"]
        spotify_track_id = song_spotify_url.split("/")[-1]
        song_spotify_id_dict[song_name] = spotify_track_id

    ## There are duclicate entries for song names. Correct the song count
    #song_count = len(song_spotify_id_dict)

    return song_spotify_id_dict

In [55]:
tracks_df = pd.DataFrame(columns=["Song_title","Spotify_track_id", "Artist"])

In [58]:
for i in range(0, len(df_artists)):
    track_ids = scrape_songs_url(df_artists.url_songs[i])

    data_dict = {}
    data_dict["Song_title"] = list(track_ids.keys())
    data_dict["Spotify_track_id"] = list(track_ids.values())

    sub_df = pd.DataFrame(data_dict)
    sub_df["Artist"] = df_artists.Artist[i]

    tracks_df = pd.concat([tracks_df, sub_df])

    
    if i % 300 == 0:
        print(f"{i/3000*100} % done")

        
    

60.0 % done
70.0 % done
80.0 % done
90.0 % done


In [59]:
tracks_df = tracks_df.reset_index(drop=True)

In [60]:
len(tracks_df)

513325

In [63]:
tracks_df.Song_title.nunique()

379536

In [64]:
tracks_df.Artist.nunique()

2998

In [65]:
tracks_df.Spotify_track_id.nunique()

460890

In [66]:
df_dub = tracks_df.drop_duplicates()
len(df_dub)

513325

In [61]:
tracks_df.sample(10)

,Song_title,Spotify_track_id,Artist
410288,Bienvenidos a Jalisco,3UoM05HDtbGb1jw029Hox3,Grupo Arriesgado
490724,Creepin',0G6bhqsZpxapSAoKLjHK86,Bootsy Collins
497345,Di Ujung Malam,33LlJCqwuyvSB78SYK8AWM,Payung Teduh
287053,Contigo,5ye0hEA5G6Zfhz2Ejx6NIo,Antonio Aguilar
227062,Vamo A Darle Lele,5E7tY6dPkEntmpRaH3rPsx,Guaynaa
250321,¿Qué Estoy Haciendo Aquí?,6sMK5NZtcziMoEwKXBlQOL,Joaquín Sabina
294424,Back To Black,44Z41mjp6AgW2NaXD0OIaP,BENEE
312082,Nee Kougililo,4GXLCERrKB4lEiaFXyEC0m,S. P. Balasubrahmanyam
129664,Hard White (Up In The Club),36Y4bI3yn16ZgNslzHc4nj,Lil Jon
114121,Kapitel 12: Mission Alex (Folge 92),12EuYDq5RmJRQEkgir8Smm,Bibi und Tina


In [62]:
tracks_df.to_csv("../data/spotify_track_ids.csv")

In [68]:
from langdetect import detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException

# Ensure the results are consistent
DetectorFactory.seed = 0

def is_english(text):
    try:
        # Detect the language of the text
        language = detect(text)
        # Check if the detected language is English
        return language == 'en'
    except LangDetectException:
        # If the text is too short or can't be detected, return False
        return False

In [69]:
tracks_df["is_english"] = tracks_df["Song_title"].apply(lambda x: is_english(x))

In [ ]:
tracks_df.to_csv("..data/spotify_track_ids_english.csv")